In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
# this is a way to merge two datasets on the basis of title
# as per shape ispe 24 columns hone chahiye lekiin since humne title ke basis pe add kiya h do title do baar add nahi hoga
movies = movies.merge(credits,on='title')

In [ ]:
movies.shape

In [ ]:
movies.head(1)

In [ ]:
movies = movies[['movie_id', 'cast', 'crew','overview','title','genres' , 'keywords']]

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)
# jyadda bda data nahi h isliye sirf drop krdiya hai 
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres
# its all in the form of dictionary we need to bring it in the form of array

In [ ]:
# now we will create a function jisme hum loop lgaenge
def convert(obj):
    L =[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
# agar yeh function lgate h toh error aaega kyuki yeh list of strings h isliye ek function aata h
# literal_eval krke woh lgaenge

In [ ]:
movies['genres']= movies['genres'].apply(convert)


In [ ]:
movies.head(1)

In [ ]:
# same thing on keywords column
movies.iloc[0].keywords

In [ ]:
movies['keywords']= movies['keywords'].apply(convert)

In [ ]:
movies.head(1)

In [ ]:
movies.iloc[0].cast

In [ ]:
# we are doing this to take only first 3 dictionaries,, first 3 names only
def convert3(obj):
    L =[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break;
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies.head(1)

In [ ]:
movies.iloc[0].crew

In [ ]:
# for crew jisme job ka value is only director
def fetch_director(obj):
    L =[]
    for i in ast.literal_eval(obj):
        if i['job']== 'Director':
            L.append(i['name'])
            break;
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head(1)

In [ ]:
movies['overview'][0]
# yeh ek string h isko hum list me convert krenge

In [ ]:
# yeh krne se ek pura sentence alag alag string me chala jata haias a List
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(1)

In [ ]:
# hum ab yeh list ko concatinate krenge aur fir yeh list ko vapas string me convert krdenge fir woh bda sa paragraph bn jaega and we will use that as a tag
# now keywords me jo bhi spaces hai woh bhi hatana pdega nahi toh woh name or surname ko alag consider krega, yeh cheez hum har jagah krenge
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head(1)

In [ ]:
movies['tag'] = movies['overview'] + movies['cast'] + movies['genres'] + movies['keywords'] + movies['crew']

In [ ]:
df = movies[['movie_id', 'title', 'tag']]
df.head()

In [ ]:
df['tag'] = df['tag'].apply(lambda x: " ".join(x))
# df['tag']

In [ ]:
# sabko lower case me daaldo
df['tag'] = df['tag'].apply(lambda x: x.lower())

# df['tag'][0]

#we need to convert texts into vectors and then somehow choose the closet vectors for the output this is known as text vectorization , we using bag of words this technique works like : first you combine all the tags for eg you wil have 5lakhs words when you combine and then you combine all the tags and it becomes the large text and usme 5000 most common words nikalnege (eh calculate kroge) uske baad hum vapas se single tag lenge and for example w1 = action hai and woh 10 baar use kiya hai ause hi top 5000 words nikal lenge and woh kitni baar use hue h 
yeh hum krne ke baad har ek movie ek liye krenge fir will do same thing for second movie and so on
#during this we will remove stop words like are,am,from,to etc



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
# to change into munpy array
cv.fit_transform(df['tag']).toarray().shape

In [ ]:
vectors = cv.fit_transform(df['tag']).toarray()
# humne 5000 words liye h ab isme mostly 0 aaega kyuki ek tag me sab 5000 words toh nahi rhenge na
vectors

In [ ]:
#yeh woh 5000 words hai jo humne select kre hai 
# cv.get_feature_names()
# yeh stemming ke baad krenge

In [ ]:
# applying steming kyuki there are various similar words like action , actions , or love , loved, loving yeh steming isko love me convert krdega teeno ko
# !pip install nltk
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
# making a fucntion
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
# ps.stem("loved")
# ps.stem("loving") same output LOVE


In [ ]:
stem("'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. samworthington zoesaldana sigourneyweaver action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d jamescameron'")
# now you will see some words meaning has been changed

In [ ]:
df['tag'] = df['tag'].apply(stem)

In [ ]:
df.head(1)

In [ ]:
 cv.get_feature_names()

In [ ]:
#we will use cosine distance for similarity , jtna thetha chota h utni similarity jyadda and vica vera
# cosine distance bhhi hota h
from sklearn.metrics.pairwise import cosine_similarity
#  iska mtlb pheli movie ka baaki 4806 movie ke sath ka distance nikalna and same with rest of the movies
similarity = cosine_similarity(vectors)

In [ ]:
# yeh pheli movie ka baaki movie ke sath similarity btarha h 
similarity[1]

In [ ]:
# ab ise andar via movie index se hum similarity me ghusenge, 
# for example avatar ka 1 h index toh hum similarity matrix me gaye 
# and usme hum baaki 4806 ke sath similarity match krenge 
# and unko sort krenge and top 5 nikal lenge
# how to find index of a particulr movie
df[df['title'] == 'Batman Begins'].index
df[df['title'] == 'Batman Begins'].index[0]
# ab humko sort bhi krna h woh kaise krenge using sorting fucntion lekin 
# woh krenge toh mvoies ka index hil jaega isliye enumerate use krenge jisse movie ke sath value change ho
# iska mtlb 0 movie ka distance 1st movie se itna h and so on ab isko list me convert krenge and then sorting
list(enumerate(similarity[0]))

In [ ]:
# iska mtlb hume first wale se sorting nahi krna h second se krna h 
# sorted(list(enumerate(similarity[0])), reverse = True,key = lambda x:x[1])
sorted(list(enumerate(similarity[119])), reverse = True,key = lambda x:x[1])[1:6]
# ab 1216 similar movie h hmari avatar movie se
    

In [ ]:
def recommend(movie):
    movie_index = df[df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse = True,key = lambda x:x[1])[1:6]
    
    for i in movies_list:
#     iska mtlb five movies DEGA
        print(df.iloc[i[0]].title)

In [ ]:
recommend('Batman Begins')